In [1]:
!pip install -U tensorflow-gpu==2.0.0 pyarrow==0.14.1 tfx==0.15.0 tensorboard==2.0.0 grpcio==1.24.3 apache-beam==2.16
#!pip install -U tensorflow==2.0.0 pyarrow==0.14.1 tfx==0.15.0 tensorboard==2.0.0 grpcio==1.24.3 apache-beam==2.16

#!pip install -U tensorflow==2.1.0 tfx pyarrow==0.15.1 tensorboard==2.1.0 apache-beam==2.17.0 grpcio==1.25 ml-metadata==0.21.0 absl-py==0.8.0
#!pip install -U tensorflow-gpu==2.1.0 tfx pyarrow==0.15.1 tensorboard==2.1.0 apache-beam==2.17.0 grpcio==1.25 ml-metadata==0.21.0 absl-py==0.8.0

!pip freeze | grep -e tensorflow -e tfx -e pyarrow

Requirement already up-to-date: tensorflow-gpu==2.0.0 in /home/denis/anaconda3/envs/tfx_env/lib/python3.7/site-packages (2.0.0)
Requirement already up-to-date: pyarrow==0.14.1 in /home/denis/anaconda3/envs/tfx_env/lib/python3.7/site-packages (0.14.1)
Requirement already up-to-date: tfx==0.15.0 in /home/denis/anaconda3/envs/tfx_env/lib/python3.7/site-packages (0.15.0)
Requirement already up-to-date: tensorboard==2.0.0 in /home/denis/anaconda3/envs/tfx_env/lib/python3.7/site-packages (2.0.0)
Requirement already up-to-date: grpcio==1.24.3 in /home/denis/anaconda3/envs/tfx_env/lib/python3.7/site-packages (1.24.3)
Requirement already up-to-date: apache-beam==2.16 in /home/denis/anaconda3/envs/tfx_env/lib/python3.7/site-packages (2.16.0)


pyarrow==0.14.1
tensorflow==2.0.0
tensorflow-data-validation==0.15.0
tensorflow-estimator==2.0.1
tensorflow-gpu==2.0.0
tensorflow-metadata==0.15.2
tensorflow-model-analysis==0.15.4
tensorflow-serving-api==2.1.0
tensorflow-transform==0.15.0
tfx==0.15.0
tfx-bsl==0.15.3


In [1]:
import os
import tempfile
import urllib
import pprint
pp = pprint.PrettyPrinter()

import absl
import tensorflow as tf
import tfx


tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

from  tfx.proto import example_gen_pb2

import absl
import tensorflow as tf
import tfx
from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import ModelValidator
from tfx.components import Pusher
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import evaluator_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.proto.evaluator_pb2 import SingleSlicingSpec
from tfx.utils.dsl_utils import external_input
import tensorflow_data_validation as tfdv

In [2]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.0.0
TFX version: 0.15.0


Use cell magic %%skip_for_export to denote cells to skip during export to pipeline.

In [3]:
%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

## Set up pipeline paths and logging

In [4]:
# Set up paths.
#_tfx_root = tfx.__path__[0]
_tfx_root = os.getcwd()

_stock_root = os.path.join(_tfx_root, 'data/stock_pipeline')
# Path which can be listened to by the model server.  Pusher will output the
# trained model here.
#_serving_model_dir = os.path.join(tempfile.mkdtemp(), 'serving_model/stock_simple')
_serving_model_dir = os.path.join(_tfx_root, 'serving_model/stock_simple')

# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO)

_data_root = os.path.join(_tfx_root, 'data/input/small')


## Create the InteractiveContext

In [5]:
# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.
# To use your own pipeline root or database, the optional properties
# `pipeline_root` and `metadata_connection_config` may be passed to
# InteractiveContext. Calls to InteractiveContext are no-ops outside of the
# notebook.
context = InteractiveContext()


## Run TFX components interactively

Next, we construct TFX components and run each one interactively using within the interactive session to obtain ExecutionResult objects.


### ExampleGen

ExampleGen brings data into the TFX pipeline.



In [6]:
%%time
# Use the packaged CSV input data.
examples = external_input(_data_root)

# Input train split is '_data_root/train/*', eval split is '_data_root/eval/*'
# test split is '_data_root/test/*'.
# Output splits are generated one-to-one mapping from input splits.
input = example_gen_pb2.Input(splits=[
                example_gen_pb2.Input.Split(name='train', pattern='train/*'),
                example_gen_pb2.Input.Split(name='eval', pattern='eval/*')
                #example_gen_pb2.Input.Split(name='test', pattern='test/*')
            ])

# Brings data into the pipeline or otherwise joins/converts training data.
example_gen = CsvExampleGen(input=examples, input_config=input)
context.run(example_gen)


INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Registering a new execution type with id 3.
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.
INFO:absl:Using 1 process(es) for Beam pipeline execution.
INFO:absl:Processing input csv data /home/denis/ML/tfx/stocks/data/input/small/train/* to TFExample.
INFO:absl:Processing input csv data /home/denis/ML/tfx/stocks/data/input/small/eval/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


CPU times: user 23.1 s, sys: 155 ms, total: 23.3 s
Wall time: 24.5 s


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: ExamplesPath
            artifacts: [Artifact(type_name: ExamplesPath, uri: /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/CsvExampleGen/examples/1/train/, split: train, id: 2)
            Artifact(type_name: ExamplesPath, uri: /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/CsvExampleGen/examples/1/eval/, split: eval, id: 3)]
        ))

In [7]:
%%skip_for_export

for artifact in example_gen.outputs['examples'].get():
  print(artifact.split, artifact.uri)

train /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/CsvExampleGen/examples/1/train/
eval /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/CsvExampleGen/examples/1/eval/
This cell will be skipped during export to pipeline.


In [8]:
%%skip_for_export

import tensorflow_data_validation as tfdv

# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = example_gen.outputs['examples'].get()[0].uri

def show_data(_uri, _count):
    # Get the list of files in this directory (all compressed TFRecord files)
    tfrecord_filenames = [os.path.join(_uri, name)
                          for name in os.listdir(_uri)]

    # Create a `TFRecordDataset` to read these files
    dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
    decoder = tfdv.TFExampleDecoder()

    # Iterate over the first 3 records and decode them using a TFExampleDecoder
    for tfrecord in dataset.take(_count):
      serialized_example = tfrecord.numpy()
      example = decoder.decode(serialized_example)
      pp.pprint(example)
    
show_data(train_uri, 3)

{'Nr': array([0]),
 'close': array([114.56], dtype=float32),
 'cross_idx': array([0.], dtype=float32),
 'date': array([b'2018-01-02'], dtype=object),
 'dir': array([0.], dtype=float32),
 'high': array([114.6025], dtype=float32),
 'low': array([114.54], dtype=float32),
 'open': array([114.6025], dtype=float32),
 'sum_vol': array([1119]),
 'time': array([b'09:30:38'], dtype=object)}
{'Nr': array([6]),
 'close': array([114.61], dtype=float32),
 'cross_idx': array([0.], dtype=float32),
 'date': array([b'2018-01-02'], dtype=object),
 'dir': array([0.], dtype=float32),
 'high': array([114.61], dtype=float32),
 'low': array([114.558], dtype=float32),
 'open': array([114.56], dtype=float32),
 'sum_vol': array([1017]),
 'time': array([b'09:31:54'], dtype=object)}
{'Nr': array([11]),
 'close': array([114.62], dtype=float32),
 'cross_idx': array([0.], dtype=float32),
 'date': array([b'2018-01-02'], dtype=object),
 'dir': array([0.], dtype=float32),
 'high': array([114.62], dtype=float32),
 'low':

### StatisticsGen (using Tensorflow Data Validation)

In [9]:
%%time
# Computes statistics over data for visualization and example validation.
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)

INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Registering a new execution type with id 5.
INFO:absl:Running executor for StatisticsGen
INFO:absl:Using 1 process(es) for Beam pipeline execution.
INFO:absl:Generating statistics for split train
INFO:absl:Statistics for split train written to /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/StatisticsGen/output/2/train/.
INFO:absl:Generating statistics for split eval
INFO:absl:Statistics for split eval written to /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/StatisticsGen/output/2/eval/.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


CPU times: user 7.93 s, sys: 182 ms, total: 8.11 s
Wall time: 8.21 s


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        output: Channel(
            type_name: ExampleStatisticsPath
            artifacts: [Artifact(type_name: ExampleStatisticsPath, uri: /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/StatisticsGen/output/2/train/, split: train, id: 4)
            Artifact(type_name: ExampleStatisticsPath, uri: /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/StatisticsGen/output/2/eval/, split: eval, id: 5)]
        ))

In [10]:
%%skip_for_export

context.show(statistics_gen.outputs['statistics'])

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


This cell will be skipped during export to pipeline.



## SchemaGen (using Tensorflow Data Validation)

SchemaGen generates a schema for your data based on computed statistics. This component also uses the Tensorflow Data Validation library.


In [11]:
# Generates schema based on statistics files.
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'], infer_feature_shape=False)
context.run(schema_gen)

INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Registering a new execution type with id 7.
INFO:absl:Running executor for SchemaGen
INFO:absl:Infering schema from statistics.
INFO:absl:Schema written to /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/SchemaGen/output/3/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        output: Channel(
            type_name: SchemaPath
            artifacts: [Artifact(type_name: SchemaPath, uri: /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/SchemaGen/output/3/, split: , id: 6)]
        ))

In [12]:
schema_gen.outputs['schema']

Channel(
    type_name: SchemaPath
    artifacts: [Artifact(type_name: SchemaPath, uri: /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/SchemaGen/output/3/, split: , id: 6)]
)

In [13]:
%%skip_for_export
#Visualize the inferred schema
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'date',BYTES,required,single,-
'time',BYTES,required,single,-
'Nr',INT,required,single,-
'close',FLOAT,required,single,-
'cross_idx',FLOAT,required,single,-
'dir',FLOAT,required,single,-
'high',FLOAT,required,single,-
'low',FLOAT,required,single,-
'open',FLOAT,required,single,-


This cell will be skipped during export to pipeline.



## ExampleValidator

ExampleValidator performs anomaly detection based on computed statistics and your data schema.


In [14]:
# Performs anomaly detection based on statistics and data schema.
example_validator = ExampleValidator(statistics=statistics_gen.outputs['statistics'], 
                                     schema=schema_gen.outputs['schema'])
context.run(example_validator)

INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Registering a new execution type with id 9.
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics.
INFO:absl:Validation complete. Anomalies written to /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/ExampleValidator/output/4/.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        output: Channel(
            type_name: ExampleValidationPath
            artifacts: [Artifact(type_name: ExampleValidationPath, uri: /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/ExampleValidator/output/4/, split: , id: 7)]
        ))

In [15]:
%%skip_for_export

context.show(example_validator.outputs['anomalies'])

This cell will be skipped during export to pipeline.


## Transform

Transform performs data transformations and feature engineering which are kept in sync for training and serving.


In [16]:
_stock_constants_module_file = 'stock_constants.py'

In [17]:
%%skip_for_export
%%writefile {_stock_constants_module_file}

DENSE_FLOAT_FEATURE_KEYS = ['close', 'open', 'high', 'low', 'sum_vol', 'Nr']

# Number of buckets used by tf.transform for encoding each feature.
#FEATURE_BUCKET_COUNT = 10

#BUCKET_FEATURE_KEYS = [
#    'time'
#]

MAX_CATEGORICAL_FEATURE_VALUES = [10000]
CATEGORICAL_FEATURE_KEYS = [
    #'date', 
    #'time'
]


# Keys
LABEL_KEY = 'dir'

def transformed_name(key):
  return key + '_xf'

Overwriting stock_constants.py
This cell will be skipped during export to pipeline.


In [18]:
_stock_transform_module_file = 'stock_transform.py'

In [19]:
%%skip_for_export
%%writefile {_stock_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

import stock_constants

_DENSE_FLOAT_FEATURE_KEYS = stock_constants.DENSE_FLOAT_FEATURE_KEYS
_CATEGORICAL_FEATURE_KEYS = stock_constants.CATEGORICAL_FEATURE_KEYS
#_FEATURE_BUCKET_COUNT = stock_constants.FEATURE_BUCKET_COUNT
#_BUCKET_FEATURE_KEYS = stock_constants.BUCKET_FEATURE_KEYS
_LABEL_KEY = stock_constants.LABEL_KEY
_transformed_name = stock_constants.transformed_name

def preprocessing_fn(inputs):
  """tf.transform's callback function for preprocessing inputs.

  Args:
    inputs: map from feature keys to raw not-yet-transformed features.

  Returns:
    Map from string feature key to transformed feature operations.
  """
  outputs = {}
  
  for key in _DENSE_FLOAT_FEATURE_KEYS:
    outputs[_transformed_name(key)] = tft.scale_to_z_score(
        _fill_in_missing(inputs[key]))
    
  for key in _CATEGORICAL_FEATURE_KEYS:
    outputs[_transformed_name(key)] = _fill_in_missing(inputs[key])

    
  dirs = _fill_in_missing(inputs[_LABEL_KEY])
  lb = tf.cast(_fill_in_missing(inputs[_LABEL_KEY]), tf.int64)
    
  outputs[_transformed_name(_LABEL_KEY)] = tf.where(
      tf.equal(lb,-1),
      tf.constant(2, dtype=tf.int64),
      lb
  )

  return outputs

def _fill_in_missing(x):
  """Replace missing values in a SparseTensor.
  Fills in missing values of `x` with '' or 0, and converts to a dense tensor.
  Args:
    x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 1
      in the second dimension.
  Returns:
    A rank 1 tensor where missing values of `x` have been filled in.
  """
  default_value = '' if x.dtype == tf.string else 0
  return tf.squeeze(
      tf.sparse.to_dense(
          tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
          default_value),
      axis=1)

Overwriting stock_transform.py
This cell will be skipped during export to pipeline.


#### Run the Transform component


In [20]:
# Performs transformations and feature engineering in training and serving.
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_stock_transform_module_file))
context.run(transform)

INFO:absl:Running driver for Transform
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Registering a new execution type with id 11.
INFO:absl:Running executor for Transform


Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`


INFO:absl:Using 1 process(es) for Beam pipeline execution.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/Transform/transform_output/5/.temp_path/tftransform_tmp/b13763399d1943acb56811e5f20979d2/saved_model.pb
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/Transform/transform_output/5/.temp_path/tftransform_tmp/e01236788d8440c281038a63b5c8bb54/saved_model.pb
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to writ

INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_output: Channel(
            type_name: TransformPath
            artifacts: [Artifact(type_name: TransformPath, uri: /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/Transform/transform_output/5/, split: , id: 8)]
        )
        transformed_examples: Channel(
            type_name: ExamplesPath
            artifacts: [Artifact(type_name: ExamplesPath, uri: /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/Transform/transformed_examples/5/train/, split: train, id: 9)
            Artifact(type_name: ExamplesPath, uri: /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/Transform/transformed_examples/5/eval/, split: eval, id: 10)]
        ))

In [21]:
context.show(transform.outputs)

{'transform_output': Channel(
    type_name: TransformPath
    artifacts: [Artifact(type_name: TransformPath, uri: /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/Transform/transform_output/5/, split: , id: 8)]
), 'transformed_examples': Channel(
    type_name: ExamplesPath
    artifacts: [Artifact(type_name: ExamplesPath, uri: /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/Transform/transformed_examples/5/train/, split: train, id: 9)
    Artifact(type_name: ExamplesPath, uri: /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/Transform/transformed_examples/5/eval/, split: eval, id: 10)]
)}

In [22]:
%%skip_for_export
train_uri = transform.outputs['transform_output'].get()[0].uri
os.listdir(train_uri)

['transformed_metadata', 'metadata', 'transform_fn']

This cell will be skipped during export to pipeline.


In [23]:
%%skip_for_export
# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = transform.outputs['transformed_examples'].get()[1].uri
show_data(train_uri, 3)


{'Nr_xf': array([-0.83789784], dtype=float32),
 'close_xf': array([1.1836962], dtype=float32),
 'dir_xf': array([0]),
 'high_xf': array([1.1997468], dtype=float32),
 'low_xf': array([1.1709853], dtype=float32),
 'open_xf': array([1.2030784], dtype=float32),
 'sum_vol_xf': array([-0.0314966], dtype=float32)}
{'Nr_xf': array([-0.83471894], dtype=float32),
 'close_xf': array([1.1675998], dtype=float32),
 'dir_xf': array([0]),
 'high_xf': array([1.1707596], dtype=float32),
 'low_xf': array([1.1709853], dtype=float32),
 'open_xf': array([1.1741068], dtype=float32),
 'sum_vol_xf': array([0.11328834], dtype=float32)}
{'Nr_xf': array([-0.83352685], dtype=float32),
 'close_xf': array([1.1708195], dtype=float32),
 'dir_xf': array([0]),
 'high_xf': array([1.1772004], dtype=float32),
 'low_xf': array([1.1718863], dtype=float32),
 'open_xf': array([1.1708869], dtype=float32),
 'sum_vol_xf': array([-0.0071857], dtype=float32)}
This cell will be skipped during export to pipeline.


In [24]:
transform.outputs['transform_graph']

Channel(
    type_name: TransformPath
    artifacts: [Artifact(type_name: TransformPath, uri: /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/Transform/transform_output/5/, split: , id: 8)]
)

## Trainer
The Trainer component will train a model that you define in TensorFlow (either using the Estimator API or the Keras API with model_to_estimator).

In [25]:
_stock_trainer_module_file = 'stock_trainer.py'

In [27]:
%%skip_for_export
%%writefile {_stock_trainer_module_file}

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import schema_utils

import stock_constants

_DENSE_FLOAT_FEATURE_KEYS = stock_constants.DENSE_FLOAT_FEATURE_KEYS
_MAX_CATEGORICAL_FEATURE_VALUES = stock_constants.MAX_CATEGORICAL_FEATURE_VALUES
_CATEGORICAL_FEATURE_KEYS = stock_constants.CATEGORICAL_FEATURE_KEYS
#_FEATURE_BUCKET_COUNT = stock_constants.FEATURE_BUCKET_COUNT
#_BUCKET_FEATURE_KEYS = stock_constants.BUCKET_FEATURE_KEYS
_LABEL_KEY = stock_constants.LABEL_KEY
_transformed_name = stock_constants.transformed_name


def _transformed_names(keys):
  return [_transformed_name(key) for key in keys]


# Tf.Transform considers these features as "raw"
def _get_raw_feature_spec(schema):
  return schema_utils.schema_as_feature_spec(schema).feature_spec


def _gzip_reader_fn(filenames):
  """Small utility returning a record reader that can read gzip'ed files."""
  return tf.data.TFRecordDataset(
      filenames,
      compression_type='GZIP')


def _build_estimator(config, hidden_units=None, warm_start_from=None):
  """Build an estimator for predicting 
  Args:
    config: tf.estimator.RunConfig defining the runtime environment for the
      estimator (including model_dir).
    hidden_units: [int], the layer sizes of the DNN (input layer first)
    warm_start_from: Optional directory to warm start from.
  Returns:
    A dict of the following:
      - estimator: The estimator that will be used for training and eval.
      - train_spec: Spec for training.
      - eval_spec: Spec for eval.
      - eval_input_receiver_fn: Input function for eval.
  """
  real_valued_columns = [
      tf.feature_column.numeric_column(key, shape=())
      for key in _transformed_names(_DENSE_FLOAT_FEATURE_KEYS)
  ]

  categorical_columns = [
      tf.feature_column.categorical_column_with_hash_bucket(  # pylint: disable=g-complex-comprehension
          key,
          hash_bucket_size=num_buckets) for key, num_buckets in zip(
              _transformed_names(_CATEGORICAL_FEATURE_KEYS),
              _MAX_CATEGORICAL_FEATURE_VALUES)
  ]    
  
  used_columns =[
      real_valued_columns, 
      #tf.feature_column.embedding_column(categorical_columns,10)
  ]
  weight_column = tf.feature_column.numeric_column('weight')
    
  my_head = tf.estimator.MultiClassHead(n_classes=3)
    
  my_estimator = tf.estimator.DNNClassifier(
    #head=my_head,
    config=config,
    feature_columns=used_columns,
    hidden_units=hidden_units or [10, 5],
    warm_start_from=warm_start_from,
    weight_column=weight_column,
    n_classes=3)    
    
  #return my_estimator
  return tf.estimator.DNNLinearCombinedEstimator(
      head=my_head,
      config=config,
      linear_feature_columns=categorical_columns,
      dnn_feature_columns=real_valued_columns,
      dnn_hidden_units=hidden_units or [10, 15, 5],
      dnn_dropout=0.5,
  #    warm_start_from=warm_start_from
  )


def _keras_model_builder():
  """Creates a DNN Keras model  for classifying iris data.
  Returns:
    A keras Model.
  """
#  real_valued_columns = [
#      tf.feature_column.numeric_column(key, shape=())
#      for key in _transformed_names(_DENSE_FLOAT_FEATURE_KEYS)
#  ]

#  categorical_columns = [
#      tf.feature_column.categorical_column_with_hash_bucket(  # pylint: disable=g-complex-comprehension
#          key,
#          hash_bucket_size=num_buckets) for key, num_buckets in zip(
#              _transformed_names(_CATEGORICAL_FEATURE_KEYS),
#              _MAX_CATEGORICAL_FEATURE_VALUES)
#  ]    
  
  feature_columns = []
  feature_layer_inputs = {}
    
  for key in _transformed_names(_DENSE_FLOAT_FEATURE_KEYS):
    feature_columns.append(tf.feature_column.numeric_column(key))
    feature_layer_inputs[key] = tf.keras.Input(shape=(1,), name=key)
    
  for key, num_buckets in zip(_transformed_names(_CATEGORICAL_FEATURE_KEYS),
                              _MAX_CATEGORICAL_FEATURE_VALUES):
    feature_columns.append(tf.feature_column.indicator_column(
                           tf.feature_column.categorical_column_with_hash_bucket(  # pylint: disable=g-complex-comprehension
                                                                     key,
                                                                     hash_bucket_size=num_buckets)))
  #  feature_layer_inputs[key] = tf.keras.Input(shape=(1,), name=key)
    feature_layer_inputs[key] = tf.keras.Input(shape=(1,), name=key, dtype=tf.string)
    

    
  #_FEATURE_KEYS = _transformed_names(_DENSE_FLOAT_FEATURE_KEYS) #+ _CATEGORICAL_FEATURE_KEYS)
  #_all_feature_columns = real_valued_columns

  opt = tf.keras.optimizers
  l = tf.keras.layers
    
  #feature_layer_inputs = [l.Input(shape=(1,), name=f) for f in _FEATURE_KEYS]
#  feature_columns_input = tf.keras.layers.DenseFeatures(_all_feature_columns)
  
  feature_columns_input = tf.keras.layers.DenseFeatures(feature_columns)
  feature_layer_outputs = feature_columns_input(feature_layer_inputs)



  d1 = l.Dense(8, activation='relu')(feature_layer_outputs)
  d1 = l.Dense(5, activation='relu')(d1)
  output = l.Dense(3, activation='softmax')(d1)

  model = tf.keras.Model(inputs=[v for v in feature_layer_inputs.values()], outputs=output)
  
  model.compile(
      loss='sparse_categorical_crossentropy',
      optimizer=opt.Adam(lr=0.001),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')])
  absl.logging.info(model.summary())

  return model


def _build_estimator_Keras_Model(config, hidden_units=None, warm_start_from=None):
  print("---------------------------------> _build_estimator_Keras_Model")
    
  my_estimator = tf.keras.estimator.model_to_estimator(
      keras_model=_keras_model_builder(), config=config)
    
  return my_estimator




def _example_serving_receiver_fn(tf_transform_graph, schema):
  print("---------------------------------> _example_serving_receiver_fn")
  """Build the serving in inputs.
  Args:
    tf_transform_graph: A TFTransformOutput.
    schema: the schema of the input data.
  Returns:
    Tensorflow graph which parses examples, applying tf-transform to them.
  """
  raw_feature_spec = _get_raw_feature_spec(schema)
  raw_feature_spec.pop(_LABEL_KEY)
    
  print(raw_feature_spec)

  raw_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
      raw_feature_spec, default_batch_size=None)
  serving_input_receiver = raw_input_fn()

  transformed_features = tf_transform_graph.transform_raw_features(
      serving_input_receiver.features)

  #print(transformed_features)
  transformed_features.pop(_transformed_name(_LABEL_KEY))
  #print('REMOVE DIR', transformed_features)
  
  return tf.estimator.export.ServingInputReceiver(
      transformed_features, serving_input_receiver.receiver_tensors)


def _eval_input_receiver_fn(tf_transform_graph, schema):
  print("---------------------------------> _eval_input_receiver_fn")
  """Build everything needed for the tf-model-analysis to run the model.
  Args:
    tf_transform_graph: A TFTransformOutput.
    schema: the schema of the input data.
  Returns:
    EvalInputReceiver function, which contains:
      - Tensorflow graph which parses raw untransformed features, applies the
        tf-transform preprocessing operators.
      - Set of raw, untransformed features.
      - Label against which predictions will be compared.
  """
    
  # Notice that the inputs are raw features, not transformed features here.
#  raw_feature_spec = _get_raw_feature_spec(schema)

#  raw_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
#      raw_feature_spec, default_batch_size=None)
#  serving_input_receiver = raw_input_fn()

#  transformed_features = tf_transform_graph.transform_raw_features(
#      serving_input_receiver.features)
    
#  labels = transformed_features.pop(_transformed_name(_LABEL_KEY))

   
#  print(transformed_features)
    
#  return tfma.export.EvalInputReceiver(
#      features=transformed_features,
#      labels=labels,
#      receiver_tensors=serving_input_receiver.receiver_tensors)


    
    
 # Notice that the inputs are raw features, not transformed features here.
  raw_feature_spec = _get_raw_feature_spec(schema)

  serialized_tf_example = tf.compat.v1.placeholder(
      dtype=tf.string, shape=[None], name='input_example_tensor')

  # Add a parse_example operator to the tensorflow graph, which will parse
  # raw, untransformed, tf examples.
  features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)

  print(list(features.keys()))
  # Now that we have our raw examples, process them through the tf-transform
  # function computed during the preprocessing step.
  transformed_features = tf_transform_graph.transform_raw_features(
      features)
    
  print(list(transformed_features.keys()))

  # The key name MUST be 'examples'.
  receiver_tensors = {'examples': serialized_tf_example}

  print(receiver_tensors)

  # NOTE: Model is driven by transformed features (since training works on the
  # materialized output of TFT, but slicing will happen on raw features.
  features.update(transformed_features)

  #for x in ['close', 'open', 'high', 'low', 'sum_vol', 'Nr', 'cross_idx', 'time', 'date','dir','dir_xf']:
  #  features.pop(x)

  print(features)
  #print(transformed_features)

  return tfma.export.EvalInputReceiver(
      features=features,
      receiver_tensors=receiver_tensors,
      labels=transformed_features[_transformed_name(_LABEL_KEY)])


def _input_fn(filenames, tf_transform_graph, batch_size=200):
  print("---------------------------------> _input_fn")
  """Generates features and labels for training or evaluation.
  Args:
    filenames: [str] list of CSV files to read data from.
    tf_transform_graph: A TFTransformOutput.
    batch_size: int First dimension size of the Tensors returned by input_fn
  Returns:
    A (features, indices) tuple where features is a dictionary of
      Tensors, and indices is a single Tensor of label indices.
  """
  transformed_feature_spec = (
      tf_transform_graph.transformed_feature_spec().copy())

  dataset = tf.data.experimental.make_batched_features_dataset(
      filenames, batch_size, transformed_feature_spec, reader=_gzip_reader_fn)

  transformed_features = dataset.make_one_shot_iterator().get_next()
    
  # We pop the label because we do not want to use it as a feature while we're
  # training.
    
  return transformed_features, transformed_features.pop(
      _transformed_name(_LABEL_KEY))


# TFX will call this function
def trainer_fn(hparams, schema):
  print("---------------------------------> trainer_fn")
  """Build the estimator using the high level API.
  Args:
    hparams: Holds hyperparameters used to train the model as name/value pairs.
    schema: Holds the schema of the training examples.
  Returns:
    A dict of the following:
      - estimator: The estimator that will be used for training and eval.
      - train_spec: Spec for training.
      - eval_spec: Spec for eval.
      - eval_input_receiver_fn: Input function for eval.
  """
  # Number of nodes in the first layer of the DNN
  first_dnn_layer_size = 10
  num_dnn_layers = 2
  dnn_decay_factor = 0.7

  train_batch_size = 100
  eval_batch_size = 100
  
  tf_transform_output = tft.TFTransformOutput(hparams.transform_output)

  train_input_fn = lambda: _input_fn(  # pylint: disable=g-long-lambda
      hparams.train_files,
      tf_transform_output,
      batch_size=train_batch_size)

  eval_input_fn = lambda: _input_fn(  # pylint: disable=g-long-lambda
      hparams.eval_files,
      tf_transform_output,
      batch_size=eval_batch_size)

  train_spec = tf.estimator.TrainSpec(  # pylint: disable=g-long-lambda
      train_input_fn,
      max_steps=hparams.train_steps)

  serving_receiver_fn = lambda: _example_serving_receiver_fn(  # pylint: disable=g-long-lambda
      tf_transform_output, schema)

  exporter = tf.estimator.FinalExporter('stocks', serving_receiver_fn)
  eval_spec = tf.estimator.EvalSpec(
      eval_input_fn,
      steps=hparams.eval_steps,
      exporters=[exporter],
      name='stocks-eval')

  run_config = tf.estimator.RunConfig(
      save_checkpoints_steps=999, keep_checkpoint_max=1)

  run_config = run_config.replace(model_dir=hparams.serving_model_dir)

  estimator = _build_estimator(
      # Construct layers sizes with exponetial decay
      hidden_units=[
          max(2, int(first_dnn_layer_size * dnn_decay_factor**i))
          for i in range(num_dnn_layers)
      ],
      config=run_config,
      warm_start_from=hparams.warm_start_from
  )


  estimator2 = _build_estimator_Keras_Model(
      # Construct layers sizes with exponetial decay
      hidden_units=[
          max(2, int(first_dnn_layer_size * dnn_decay_factor**i))
          for i in range(num_dnn_layers)
      ],
      config=run_config,
      warm_start_from=hparams.warm_start_from)

  # Create an input receiver for TFMA processing
  receiver_fn = lambda: _eval_input_receiver_fn(  # pylint: disable=g-long-lambda
      tf_transform_output, schema)

  return {
      'estimator': estimator,
      'train_spec': train_spec,
      'eval_spec': eval_spec,
      'eval_input_receiver_fn': receiver_fn
  }

Overwriting stock_trainer.py
This cell will be skipped during export to pipeline.


### Run the Trainer component

In [28]:
# Uses user-provided Python function that implements a model using TensorFlow.
trainer = Trainer(
    module_file=os.path.abspath(_stock_trainer_module_file),
    transformed_examples=transform.outputs['transformed_examples'],
    schema=schema_gen.outputs['schema'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=trainer_pb2.TrainArgs(num_steps=10000),
    eval_args=trainer_pb2.EvalArgs(num_steps=5000))
context.run(trainer)


INFO:absl:Running driver for Trainer
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Registering a new execution type with id 13.
INFO:absl:Running executor for Trainer


---------------------------------> trainer_fn
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/Trainer/output/6/serving_model_dir', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 999, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5829ced910>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, 

INFO:absl:None


---------------------------------> _build_estimator_Keras_Model
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Nr_xf (InputLayer)              [(None, 1)]          0                                            
__________________________________________________________________________________________________
close_xf (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
high_xf (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
low_xf (InputLayer)             [(None, 1)]          0                                            
______________________________

INFO:absl:Training model.


...

INFO:tensorflow:loss = 0.4825908, step = 9900 (0.089 sec)
INFO:tensorflow:Saving checkpoints for 9990 into /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/Trainer/output/6/serving_model_dir/model.ckpt.
INFO:tensorflow:Skip the current checkpoint eval due to throttle secs (600 secs).
INFO:tensorflow:Saving checkpoints for 10000 into /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/Trainer/output/6/serving_model_dir/model.ckpt.
INFO:tensorflow:Skip the current checkpoint eval due to throttle secs (600 secs).
---------------------------------> _input_fn
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-02-12T21:29:27Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/Trainer/output/6/serving_model_dir/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evalua

INFO:absl:Training complete.  Model written to /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/Trainer/output/6/serving_model_dir
INFO:absl:Exporting eval_savedmodel for TFMA.


---------------------------------> _eval_input_receiver_fn
['Nr', 'close', 'cross_idx', 'date', 'dir', 'high', 'low', 'open', 'sum_vol', 'time']
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
['low_xf', 'dir_xf', 'Nr_xf', 'sum_vol_xf', 'open_xf', 'close_xf', 'high_xf']
{'examples': <tf.Tensor 'input_example_tensor:0' shape=(None,) dtype=string>}
{'Nr': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7f57fb8f5e50>, 'close': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7f5834084f10>, 'cross_idx': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7f5833aef710>, 'date': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7f5833aef690>, 'dir': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7f583308f850>, 'high': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7f583308fd10>, 'low': <tensorflow.python.framework.sparse_tensor.SparseTens

INFO:absl:Exported eval_savedmodel to /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/Trainer/output/6/eval_model_dir.
INFO:absl:Running publisher for Trainer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        output: Channel(
            type_name: ModelExportPath
            artifacts: [Artifact(type_name: ModelExportPath, uri: /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/Trainer/output/6/, split: , id: 11)]
        ))

#### The recommended way to install TFMA is using the PyPI package:
https://github.com/tensorflow/model-analysis#installation

### Run TFMA using the Evaluator component
Here, we first define slicing specs for analyzing our data. Next, we run TFMA using these specs to generate results.


In [29]:
# An empty slice spec means the overall slice, that is, the whole dataset.
OVERALL_SLICE_SPEC = evaluator_pb2.SingleSlicingSpec()

# Data can be sliced along a feature column
# In this case, data is sliced along feature column trip_start_hour.
FEATURE_COLUMN_SLICE_SPEC = evaluator_pb2.SingleSlicingSpec(
    column_for_slicing=['dir'])

# Data can be sliced by crossing feature columns
# In this case, slices are computed for trip_start_day x trip_start_month.
FEATURE_COLUMN_CROSS_SPEC = evaluator_pb2.SingleSlicingSpec(
    column_for_slicing=['time', 'close'])

ALL_SPECS = [
    OVERALL_SLICE_SPEC,
    FEATURE_COLUMN_SLICE_SPEC,
    FEATURE_COLUMN_CROSS_SPEC,
]


In [30]:
# Use TFMA to compute a evaluation statistics over features of a model.
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    feature_slicing_spec=evaluator_pb2.FeatureSlicingSpec(
        specs=ALL_SPECS)
)
context.run(evaluator)

INFO:absl:Running driver for Evaluator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Registering a new execution type with id 15.
INFO:absl:Running executor for Evaluator
INFO:absl:Using /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/Trainer/output/6/eval_model_dir/1581539371 for model eval.
INFO:absl:Evaluating model.
INFO:absl:Using 1 process(es) for Beam pipeline execution.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/Trainer/output/6/eval_model_dir/1581539371/variables/variables
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.get_tensor_from_tensor_info or tf.compat.v1.saved_model.get_tensor_from_tensor_info.


INFO:absl:Evaluation complete. Results written to /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/Evaluator/output/7/.
INFO:absl:Running publisher for Evaluator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Evaluator
    execution_id: 7
    outputs:
        output: Channel(
            type_name: ModelEvalPath
            artifacts: [Artifact(type_name: ModelEvalPath, uri: /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/Evaluator/output/7/, split: , id: 12)]
        ))

### Show the TFMA visualization for the default slices

In [31]:
%%skip_for_export

context.show(evaluator.outputs['evaluation'])

This cell will be skipped during export to pipeline.


In [34]:
%%skip_for_export

import tensorflow_model_analysis as tfma

PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(PATH_TO_RESULT)

This cell will be skipped during export to pipeline.


In [36]:
%%skip_for_export

# Show data sliced along feature column Nr.
tfma.view.render_slicing_metrics(tfma_result)

SlicingMetricsViewer(config={'weightedExamplesColumn': 'post_export_metrics/example_count'}, data=[{'slice': '…

This cell will be skipped during export to pipeline.


In [37]:
%%skip_for_export

# Show data sliced along feature column Nr.
tfma.view.render_slicing_metrics(tfma_result, slicing_column='dir')

SlicingMetricsViewer(config={'weightedExamplesColumn': 'post_export_metrics/example_count'}, data=[{'slice': '…

This cell will be skipped during export to pipeline.


In [38]:
%%skip_for_export

# Show metrics sliced by 'trip_start_day' x 'trip_start_month'.
tfma.view.render_slicing_metrics(
    tfma_result,
    slicing_spec=tfma.slicer.SingleSliceSpec(
        columns=['time', 'close']))



SlicingMetricsViewer(config={'weightedExamplesColumn': 'post_export_metrics/example_count'}, data=[{'slice': '…

This cell will be skipped during export to pipeline.


In [39]:
%%skip_for_export

# Show overall metrics.
tfma.view.render_slicing_metrics(tfma_result)

SlicingMetricsViewer(config={'weightedExamplesColumn': 'post_export_metrics/example_count'}, data=[{'slice': '…

This cell will be skipped during export to pipeline.


In [40]:
%%skip_for_export

# Note that we are showing the same set of results three times here. Please replace
# PATH_TO_RESULT_1 ~ 3 as more results become available.

PATH_TO_RESULT_1 = PATH_TO_RESULT
PATH_TO_RESULT_2 = PATH_TO_RESULT
PATH_TO_RESULT_3 = PATH_TO_RESULT


result_1 = tfma.load_eval_result(PATH_TO_RESULT_1)
result_2 = tfma.load_eval_result(PATH_TO_RESULT_2)
result_3 = tfma.load_eval_result(PATH_TO_RESULT_3)

eval_results = tfma.make_eval_results([result_1, result_2, result_3],
                                      tfma.constants.MODEL_CENTRIC_MODE)
tfma.view.render_time_series(eval_results, tfma.slicer.SingleSliceSpec())

TimeSeriesViewer(config={'isModelCentric': True}, data=[{'metrics': {'': {'': {'accuracy': {'doubleValue': 0.9…

This cell will be skipped during export to pipeline.


### ModelValidator
ModelValidator performs validation of your candidate model compared to a baseline.

In [41]:
# Performs quality validation of a candidate model (compared to a baseline).
model_validator = ModelValidator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'])
context.run(model_validator)

INFO:absl:Running driver for ModelValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Registering a new execution type with id 17.
INFO:absl:Resolved last blessed model None
INFO:absl:Running executor for ModelValidator
INFO:absl:Creating temp directory at /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/.temp/8/
INFO:absl:Using temp path /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/.temp/8/ for tft.beam
INFO:absl:Using /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/Trainer/output/6/ as current model.
INFO:absl:Using None as blessed model.
INFO:absl:Validating model.
INFO:absl:Using 1 process(es) for Beam pipeline execution.


INFO:tensorflow:Restoring parameters from /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/Trainer/output/6/eval_model_dir/1581539371/variables/variables


INFO:absl:all files in current_model_eval_result_path: [['eval_config.json', 'metrics', 'plots']]
INFO:absl:Current model passes threshold.
INFO:absl:No blessed model yet.
INFO:absl:Blessing result True written to /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/ModelValidator/blessing/8/.
INFO:absl:Cleaned up temp path /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/.temp/8/ on executor success.
INFO:absl:Running publisher for ModelValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ModelValidator
    execution_id: 8
    outputs:
        blessing: Channel(
            type_name: ModelBlessingPath
            artifacts: [Artifact(type_name: ModelBlessingPath, uri: /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/ModelValidator/blessing/8/, split: , id: 13)]
        ))

In [42]:
%%skip_for_export

model_validator.outputs

{'blessing': Channel(
    type_name: ModelBlessingPath
    artifacts: [Artifact(type_name: ModelBlessingPath, uri: /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/ModelValidator/blessing/8/, split: , id: 13)]
)}

This cell will be skipped during export to pipeline.


### Pusher
Pusher checks whether a model has passed validation, and if so, pushes the model to a file destination.

In [43]:
# Checks whether the model passed the validation steps and pushes the model
# to a file destination if check passed.
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=model_validator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
context.run(pusher)

INFO:absl:Running driver for Pusher
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Registering a new execution type with id 19.
INFO:absl:Running executor for Pusher
INFO:absl:Model pushing.
INFO:absl:Model version is 1581539371
INFO:absl:Model written to /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/Pusher/model_push/9/.
INFO:absl:Model written to serving path /home/denis/ML/tfx/stocks/serving_model/stock_simple/1581539371.
INFO:absl:Model pushed to /home/denis/ML/tfx/stocks/serving_model/stock_simple/1581539371.
INFO:absl:Running publisher for Pusher
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Pusher
    execution_id: 9
    outputs:
        model_push: Channel(
            type_name: ModelPushPath
            artifacts: [Artifact(type_name: ModelPushPath, uri: /tmp/tfx-interactive-2020-02-12T20_40_28.962852-e02vkx65/Pusher/model_push/9/, split: , id: 14)]
        ))

In [44]:
#@title Select a runner type. { display-mode: "form" }
_runner_type = 'beam' #@param ["beam", "airflow"]
_pipeline_name = 'stock_%s' % _runner_type

In [45]:
# TODO(USER).

# Jupyter example.
_notebook_filepath = os.path.join(os.getcwd(),'tfx_nb.ipynb')

# TODO(USER): Paths for exported pipeline.
_tfx_root = os.path.join(os.environ['HOME'], 'tfx_export')
_stock_root = os.path.join(os.environ['HOME'], 'tfx_export/stock')
_serving_model_dir = os.path.join(_stock_root, 'serving_model')
#_data_root = os.path.join(_stock_root, 'data', 'simple')
_data_root = os.path.join(_stock_root, 'data/input/small')

_pipeline_root = os.path.join(_stock_root, 'pipelines', _pipeline_name)
# Sqlite ML-metadata db path.
_metadata_path = os.path.join(_stock_root, 'metadata', _pipeline_name,
                              'metadata.db')


### Specify components in exported pipeline

In [46]:
components = [
    # TODO(USER): Specify components to be part of the exported pipeline.
    example_gen, statistics_gen, schema_gen, example_validator, transform,
    trainer, evaluator, model_validator, pusher
]

#### Save your notebook, then run the next cell to export a pipeline to run on Beam/Airflow/etc.

In [47]:
%%skip_for_export #@title Run cell to export pipeline. { display-mode: "form" }

if get_ipython().magics_manager.auto_magic:
  print('Warning: %automagic is ON. Line magics specified without the % prefix '
        'will not be scrubbed during export to pipeline.')

_pipeline_export_filepath = 'export_%s.py' % _pipeline_name
context.export_to_pipeline(notebook_filepath=_notebook_filepath,
                           export_filepath=_pipeline_export_filepath,
                           runner_type=_runner_type)

INFO:absl:Exporting contents of /home/denis/ML/tfx/stocks/tfx_nb.ipynb to export_stock_beam.py with beam runner.
INFO:absl:20 cell(s) marked with "%%skip_for_export", skipped.


This cell will be skipped during export to pipeline.


In [48]:
%%skip_for_export #@title Run cell to download exported files.

import zipfile

zip_export_path = os.path.join(
  os.environ['HOME'], 'export.zip')

print(zip_export_path)

with zipfile.ZipFile(zip_export_path, mode='w') as export_zip:
    export_zip.write(_pipeline_export_filepath)
    export_zip.write(_stock_constants_module_file)
    export_zip.write(_stock_transform_module_file)
    export_zip.write(_stock_trainer_module_file)



/home/denis/export.zip
This cell will be skipped during export to pipeline.
